In [44]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
from IPython.display import clear_output
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')

In [75]:
def get_time_spaced_traindata(data):
    data['delta_t'] = np.int64((data.index - time_range[0]).total_seconds() // 3600) + 1
    data = data.drop(columns=['station_id'])
    data.index = range(len(data))
    return data

def get_time_spaced_testdata(data):
    data['delta_t'] = np.int64((data.index - time_range[0]).total_seconds() // 3600) + 1  
    data = data.drop(columns=['station_id'])
    data.index = range(len(data))
    aq_data = data['PM25_Concentration']
    data.drop(columns=['PM25_Concentration'])
    return data,aq_data

In [76]:
fold=0 # not using for loop to avoid ram overflow
print('fold', fold)

train_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_train.csv.gz')
train_data['time'] = pd.to_datetime(train_data['time'])
train_data = train_data.set_index('time').sort_values(['time', 'station_id'])

test_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_test.csv.gz')
test_data['time'] = pd.to_datetime(test_data['time'])
test_data = test_data.set_index('time').sort_values(['time', 'station_id'])

fold 0


In [77]:
train_input = get_time_spaced_traindata(train_data)
test_input,test_output = get_time_spaced_testdata(test_data)

In [78]:
display(train_input,test_input,test_output)

,longitude,latitude,PM25_Concentration,temperature,humidity,wind_speed,weather_0.0,weather_1.0,weather_2.0,weather_3.0,...,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0,delta_t
0,116.173553,40.090679,138.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,116.205310,40.003950,89.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,116.171150,39.815128,98.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,116.136045,39.742767,109.0,19.0,61.0,11.16,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,116.397400,39.982053,88.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175195,116.628000,40.328000,208.0,18.7,80.0,1.10,0,1,0,0,...,0,0,0,0,0,0,0,0,1,8760
175196,116.220000,40.292000,87.0,21.0,65.0,3.00,1,0,0,0,...,1,0,0,0,0,0,0,0,0,8760
175197,116.911000,40.499000,147.0,17.9,80.0,0.40,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8760
175198,116.783000,39.712000,244.0,21.0,61.0,8.50,1,0,0,0,...,0,0,1,0,0,0,0,0,0,8760


,longitude,latitude,PM25_Concentration,temperature,humidity,wind_speed,weather_0.0,weather_1.0,weather_2.0,weather_3.0,...,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0,delta_t
0,116.184239,39.914409,105.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,116.287451,39.987313,87.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,116.460742,39.937119,78.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,116.483746,39.939554,74.0,20.0,56.0,7.92,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,116.406155,39.718147,86.0,19.0,62.0,4.68,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96355,116.831999,40.369999,178.0,17.9,80.0,0.40,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8760
96356,115.971999,40.453000,79.0,17.9,76.0,1.50,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8760
96357,115.988000,40.365000,80.0,17.9,76.0,1.50,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8760
96358,117.120000,40.100000,164.0,19.5,72.0,2.20,0,1,0,0,...,0,0,0,0,0,1,0,0,0,8760


0        105.0
1         87.0
2         78.0
3         74.0
4         86.0
         ...  
96355    178.0
96356     79.0
96357     80.0
96358    164.0
96359     94.0
Name: PM25_Concentration, Length: 96360, dtype: float64